In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import StratifiedKFold, KFold

In [2]:
# data 경로
path = 'data'
train_path = os.path.join(path, 'train_features.csv')
label_path = os.path.join(path, 'train_labels.csv')
test_path = os.path.join(path, 'test_features.csv')
sub_path = os. path.join(path, 'sample_submission.csv')

train = pd.read_csv(train_path)
label = pd.read_csv(label_path)
test = pd.read_csv(test_path)
sub = pd.read_csv(sub_path)

In [4]:
k_split = KFold(n_splits=5)

In [32]:
def ids_26(label):
    mask = label['label'] == 26
    ids = label.loc[mask, 'id'].tolist()
    
    return ids


def make_series(data):
    ids = data['id'].unique()
    id_data = data.groupby('id')
    series_data = []

    for i in ids:
        df = id_data.get_group(i)
        df = df.drop(['id', 'time'], axis=1)
        series_data.append(df.to_numpy())

    series_data = np.array(series_data)
    return series_data


def make_split_dataset(series_train, ids, labels):
    final_list = []
    
    k_split = KFold(n_splits=5)
    
    temp = np.array(range(3125))
    except_train_mask = np.setdiff1d(temp, ids, assume_unique=True)
    except_train = series_train[except_train_mask]
    except_label = labels[except_train_mask]
    
    train_26 = series_train[ids]

    for _, fold in k_split.split(train_26):
        temp_train = train_26[fold]
        temp_label = np.array([26] * len(temp_train))
        
        temp_train = np.concatenate([temp_train, except_train], axis=0)
        temp_label = np.concatenate([temp_label, except_label], axis=0)
        print(temp_train.shape)
        final_list.append([temp_train, temp_label])
        
    return final_list

In [33]:
series_train = make_series(train)
series_test = make_series(test)

In [34]:
ids = ids_26(label)

data_list = make_split_dataset(series_train, ids, label['label'])

(1911, 600, 6)
(1911, 600, 6)
(1911, 600, 6)
(1910, 600, 6)
(1910, 600, 6)


In [37]:
data_list[0][1]

array([26, 26, 26, ..., 17, 15,  2], dtype=int64)